In [2]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p cv2,PIL,matplotlib
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp
import numpy as np

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            for s in ['main', 'master']:
                uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
                    break
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            for s in ['/raw/main/', '/raw/master/']:
                uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:]) + '.py'
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
                    break
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass


numpy 1.19.5
sklearn 0.0
pandas 1.1.5
cv2 4.5.3
PIL 8.3.1
matplotlib 3.3.4
torch 1.8.1+cu101
torchvision 0.9.1+cu101
torchaudio not installed
tensorflow 2.6.0
tensorboard 2.6.0


In [1]:
import tensorflow as tf

In [15]:
a = tf.random.uniform((1,2,3,4), dtype=tf.float32)
a

<tf.Tensor: shape=(1, 2, 3, 4), dtype=float32, numpy=
array([[[[0.5799602 , 0.4502045 , 0.09852731, 0.341141  ],
         [0.86816907, 0.24651933, 0.8749602 , 0.5034946 ],
         [0.92279184, 0.14655364, 0.13847852, 0.62896526]],

        [[0.18332481, 0.853068  , 0.7495192 , 0.28849328],
         [0.5683292 , 0.88377106, 0.4638723 , 0.23861909],
         [0.48134518, 0.9460112 , 0.19557548, 0.5672623 ]]]],
      dtype=float32)>

In [16]:
b = tf.reduce_max(a, [1])
tf.reduce_max(b, [1])

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.92279184, 0.9460112 , 0.8749602 , 0.62896526]], dtype=float32)>

In [17]:
tf.reduce_max(a, [1,2])

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.92279184, 0.9460112 , 0.8749602 , 0.62896526]], dtype=float32)>